In [ ]:
from IPython.display import Markdown
from datetime import datetime, timedelta

import duckdb
import locale
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit
import suncalc

locale.setlocale(locale.LC_ALL, 'de_DE')

tz='Europe/Berlin'

# Create a connection to your PV database. 
# Make sure you installed the schema as described in the README of the pv project.
db = duckdb.connect('pv.db')

# Statistiken und Gedanken zur Photovoltaikanlage

In [ ]:
now = datetime.now()
Markdown(f"""
Letzte Aktualisierung: _{now.strftime('%x')}, {now.strftime('%X')}_.

Die Quelltexte, inklusiver aller SQL Abfrage sowie das Jupyiter Notebook mit dem diese Seite generiert wurde sind unter Apache 2 License in meinem PV Repository veröffentlicht: [michael-simons/pv](https://github.com/michael-simons/pv).
Diese Seite dient als kurzer Überblick über unsere Photovoltaikanlage in Aachen. Mit der Planung begannen wir im September 2022, Ende April 2023 speiste der Wechselrichter das erste Mal Strom in den Hauskreislauf und das Stromnetz, Ende Mai habe ich zum ersten Mal sinnvoll bilanzierte Werte aus dem Monitoringdevice auslesen können und auf den 2-Wegezähle und Einspeisevergütung warten wir zum jetzigen Standpunkt immer noch.
""")

## Warum diese Seite?

Ursprünglich hatte ich nicht vor, allzu viel Arbeit und Ehrgeiz in die Analyse der Daten der PV-Anlage zu stecken, weder in Charts noch irgendwelche anderen Satellitensoftware. Immerhin hatten wir doch bereits einen Energiemanager mitgekauft, der irgendwann einmal Batterie- und Wallboxsteuerung übernehmen soll. Es stellte sich aber schnell heraus, dass - wie üblich - Smartdevices nicht wirklich smart sind, Konfiguration auch für Profis schwierig ist und am Ende hab ich doch wieder selber etwas gebaut: Das aktuelle Setup liest die Daten des Wechselrichters direkt über [SunSpec Modbus](https://sunspec.org/sunspec-modbus-specifications/) aus und schreibt sie in eine [DuckDB Datenbank](https://duckdb.org). Den zu diesem Zweck programmierte  [Logger](https://github.com/michael-simons/pv/tree/main/logger) basiert auf dem [Energy systems reading toolkit](https://energy.basjes.nl) von [Niels Basjes](https://mastodon.basjes.nl/@niels). Bedankt, Niels.

DuckDB ist eine sehr coole In-Prozess Datenbank mit dem Schwerpunkt [OLAP](https://de.wikipedia.org/wiki/Online_Analytical_Processing). Das Datenbankdesign dieser Anwendung folgt einer Teilmenge des [pink database design](https://www.salvis.com/blog/2018/07/18/the-pink-database-paradigm-pinkdb/#feature_3). Anstatt alle Queries direkt in einer Anwendung zu schreiben (oder gar ein ORM Framework zu nutzen und Statistiken in der Anwendung zu berechnen), habe ich eine ganze handvoll Views erstellt. Diese repräsentieren sozusagen die öffentliche API des Schemas. Sie sind natürlich auch im Repository gespeichert und liegen im [Schema-Ordner](https://github.com/michael-simons/pv/tree/main/schema) (Alle Dateien die mit `R__` anfangen).

## Warum die Photovoltaikanlage?

Wir wohnen seit 2010 im selben Ort und unsere Bezugskosten sind bis Ende 2019 mehr oder weniger kontinuierlich, aber vorhersagbar gestiegen. Mit der Pandemie, dem Krieg in der Ukraine, den Kosten des Klimawandels und schlussendlich der daraus steigenden Inflation wurden die Preise danach immer unkalkulierbarer. Langfristig ist unser Ziel, zumindest über einen Teil des Jahres stabile Stromkosten zu haben *und* dieses Haus nachhaltiger zu bewirtschaften.

Im folgenden ein Diagramm unserer Beschaffungskosten seit 2010.

In [ ]:
buying_prices = db.sql('SELECT valid_from AS Jahr, net FROM v__buying_prices').df()
imported = db.sql("SELECT date_trunc('year', period_end) as Jahr, import FROM official_measurements ORDER by period_start ASC").df()

imported_with_price = pd.merge_asof(buying_prices, imported, on='Jahr')

fig, ax = plt.subplots()
imported_with_price.plot(x = 'Jahr', y = 'net',    ylabel='Netto Preis in ct/kWh', label='Preis', ax = ax,)
imported_with_price.plot(x = 'Jahr', y = 'import', ylabel='Bezogene Energie in kWh', label='Arbeit', ax = ax, secondary_y = True, mark_right=False)

plt.show()

Während die Menge bezogener Energie über die Jahre verhältnismäßig konst geblieben ist (2018 habe ich angefangen, 100% Remote zu Arbeite und seit ein paar Jahren spielen die Kinder mehr Computer als ich an einem arbeite, aber der Mehrbetrag ist vernachlässigbar). Der Bezugspreis hingegen ist seit 2010 um **75%** gestiegen. 

## Die Anlage

Wir haben eine Anlage mit einer Peak-Leistung von 10.53kWp aufgebaut in einer aufgeteilten Ost/West Aufstellung. Ingesamt sind 26 [Solarwatt Glass-Glass "Panel Vision AM 4.0"](https://solarwatt.canto.global/direct/document/2bp9ip7a492p51429ek8qdr706/HPyRw7XsY0A3hu2B1c6SpNGxiOc/original?content-type=application%2Fpdf&name=Datenblatt+SOLARWATT+Panel+vision+AM+4.0+pure+de.pdf) Module mit einem Peak-Output von 405Wp pro Modul verbaut. Alle Module sind "Plus-Auswahl", d.h. 405Wp sind garantiert.

* 14 in östlicher Richtung
* 12 in westlicher Richtung

Als Wechselrichter kommt ein Kaco [blueplanet 10.0 NX3 M2](https://kaco-newenergy.com/de/produkte/blueplanet-3.0-20.0-NX3-M2/) zum Einsatz, eines der wenigen Modelle die unser Installateur beziehungsweise [Solarwatt](https://www.solarwatt.de) liefern konnte. Ebenfalls von Solarwatt stammt der "Energymanager", der ist im ersten Monat soviel "Freude" bereitet hat. Immerhin, die Hotline war bemüht und hilfreich.

Der mögliche Peak-Output von 10.53kWp ist deutlich mehr als wie tatsächlich täglich verbrauchen, aber mit Hinblick auf Autarkie in der Zukunft und weitere Anwendungen, wollten wir lieber sicher gehen. Bis jetzt steht noch offen, ob wir unser mehr als 10 Jahre altes Auto mit einem elektrischen ersetzen oder eine stationäre Batterie, eine Wärmepumpe oder beides kaufen. Auf Mastodon bekamen wir schon den Tipp, die Datensammlung und Analyse wie her fortzusetzen und insbesondere auf die Einspeisung zwischen Oktober und Februar zu achten: Größer als diese Menge braucht eine Batterie nicht sein. Ich finde den Tipp super, da ich gewohnheitsmässig eher zur Überdimensionierung neige. Bis dahin gibt es die bisherigen Ergebnisse:

## Ergebnisse

### Erzeugung

In [ ]:
period = db.sql('SELECT min(measured_on) AS min, max(measured_on) AS max FROM measurements').df()
Markdown(f"""
Aktuell liegen Messungen vom **{period['min'][0].strftime('%x')}** bis zum **{period['max'][0].strftime('%x')}** vor. In dieser Zeit haben wir folgende Werte über alles ermittelt:
""")

In [ ]:
new_names = {
    'worst': 'Schlechtester Tag',
    'best': 'Bester Tag',
    'daily_avg': 'Durchschnitt',
    'daily_median': 'Median',
    'total': 'Gesamtproduktion',
    'total_yield': 'Ertrag (kWh/kWp)',
    'year': 'Jahr'
}

db.sql('SELECT * FROM v_overall_production').df().rename(columns=new_names)

mit durschnittlichen Werten pro Monat wie folgt:

In [ ]:
average_per_month = db.sql('SELECT * FROM v_average_production_per_month').df()
average_per_month['month'] = average_per_month['month'].apply(lambda x: x.strftime('%B'))
average_per_month.plot(kind='bar', x='month', xlabel='Monat', ylabel='Erzeugung in kWh', figsize=plt.figaspect(1/2))
plt.legend([], frameon=False)
plt.show()

Die Erzeugung heruntergebrochen auf einzelne Jahre:

In [ ]:
db.sql('SELECT * FROM v_yearly_production').df().rename(columns=new_names).set_index('Jahr')

In [ ]:
max_peak = db.sql('SELECT round(production) as max_peak FROM v_peaks').df()['max_peak'][1];
Markdown(f"""
Die Wahrscheinlichkeit, dass wir jemals an die tatsächlich installierte Peak-Leistung herankommen werden, ist denkbar gering. 
Der bisher höchste, gemessene Wert in einer Viertelstunde waren **{max_peak}W**.
Viel interessanter - und relevanter - ist jedoch die Tatsache, dass wir mit der gewählten Dachbelegung den ganzen Tag über einen brauchbaren Output habe. 
Der Wechselrichter startet zeitnah mit Sonnenaufgang und bereits im Mai wurde im Schnitt noch um 18:00 Uhr herum genügend Energie zum Kochen produziert. 
Das kann als Durchschnittswert pro Stunde visualisiert werden:
""")

In [ ]:
average_per_hour = db.sql('SELECT * FROM v_average_production_per_hour').df()
average_per_hour.plot(kind='barh', x='hour', xlabel='Erzeugung in kWh', ylabel='Stunde', figsize=plt.figaspect(1/2)).invert_yaxis()
plt.legend([], frameon=False)
plt.show()

Die Werte für den Tag mit der bisherigen höchsten Erzeugung sehen so aus:

In [ ]:
best_day = db.sql('SELECT * FROM v_best_performing_day').df()
lat_long = db.sql('SELECT * FROM v_place_of_installation').df()
lat, long = lat_long['lat'][0], lat_long['long'][0]

localize_tz = lambda v: v.tz_localize(tz=tz)
to_degree = lambda v: math.degrees(v) if v > 0 else 0

best_day['measured_on'] = best_day['measured_on'].apply(localize_tz)
best_day['altitude'] = pd.DataFrame(suncalc.get_position(best_day['measured_on'], long, lat)['altitude'].apply(to_degree))

_, ax = plt.subplots(figsize=plt.figaspect(1/2))
best_day.plot(x = 'measured_on', y='production', xlabel='Uhrzeit', ylabel='W', label='Erzeugung', ax = ax)
best_day.plot(x = 'measured_on', y='altitude',   xlabel='Uhrzeit ', ylabel='Sonnenstand über dem Horizont in °', label='Sonnenstand', ax = ax, secondary_y = True, mark_right=False)

sun_times = suncalc.get_times(best_day['measured_on'][0].floor('d') + timedelta(days=1), long, lat)
sun_times = {k: v.tz_localize(tz='UTC').tz_convert(tz='Europe/Berlin') for k, v in sun_times.items()}

ax.axvspan(sun_times['dawn'], sun_times['sunrise_end'], color='red', alpha=0.1)
ax.axvspan(sun_times['solar_noon'] - timedelta(minutes=5), sun_times['solar_noon'] + timedelta(minutes=5), color='yellow', alpha=0.4)
ax.axvspan(sun_times['sunset_start'], sun_times['dusk'], color='blue', alpha=0.1)

plt.show()

In [ ]:
Markdown(f"""
Die orangene Kurve gibt den Sonnenverlauf am {best_day['measured_on'][0].strftime('%x')} wieder, 
der rote Balken markiert die Zeit von Anfang der Morgendämmerung bis zum Ende des Sonnenaufgangs an diesem Tag,
der gelbe eine Phase von 10 Minuten um den wahren Mittag (dem Sonnenhöchststand) herum und
der blaue schlussendlich die Zeit vom Sonnenuntergang bis Ende der Abenddämmerung.
""")

Ich mochte die Visualisierung von [Oli](https://social.tchncs.de/@oli) im Beitrag ["Ein Jahr Photovoltaik: Tolle Dinge, die man mit SQL machen kann"](https://tonick.net/p/2022/12/ein-jahr-photovoltaik/) so sehr, dass ich sie auch haben wollte. Und zwar die durchschnittliche Erzeugung pro Monat und Stunden, quasi die Kombination der beiden oberen Balkendiagramme. Diese Heatmap sieht tatsächlich nicht nur schön aus, sie ist auch informativ und gibt direkt eine Idee, zu welchem Teil des Jahres der Eigennutzungsgrad sehr hoch sein wird und wann nicht.

DuckDB unterstützt ebenfalls ein `PIVOT`-Statement hat (und das sogar SQL-Standardkonformm, siehe [`average_production_per_month_and_hour`](https://github.com/michael-simons/pv/blob/main/schema/R__Create_view_average_production_per_month_and_hour.sql)) und Pandas kann ganz einfach eine Tabelle mit den Farbkarten aus Matplotlib einfärben:

In [ ]:
new_names = {str(i): datetime(2023, i, 1).strftime('%B') for i in range(1,13)}
new_names['hour'] = 'Stunde'
df = db.sql('SELECT * FROM v_average_production_per_month_and_hour').df().rename(columns=new_names).set_index(['Stunde']).fillna(0)
df.style.background_gradient(cmap='YlOrRd')

Die folgende Visualisierung zeigt ein Sonnenstandverlaufsdiagramm für den Ort unseres Hauses in Aachen. Zwei Dinge werden sehr schnell deutlich: Wir werden nicht viel Ertrag in den Wintermonaten haben, die Sonne wird zwischen 9 und 17 Uhr in einem für uns ungünstigen Winkel scheinen. Am kürzesten Tag des Jahres steht die Sonne am wahren Mittag genauso hoch wie sie es am längsten Tag des Jahres bereits um 8 Uhr morgens tut. 

In der Darstellung befinden sich 4 Kurven: der Verlauf der Sonnen zur Winter- und Sommersonnenwende, der Verlauf am bisher produktivsten Tag sowie in Grau ein Tag zwischen den Sonnenwendtagen als Orientierung.

Die hübschen, 8-förmigen Schleifen heissen [Analemma](https://de.wikipedia.org/wiki/Analemma). Diese Figur entsteht, wenn der Sonnenverlauf von einem fixen Punkt auf der Erde über ein Jahr täglich zur selben Zeit beobachten. Durch die Analemma können die Uhrzeiten auf den Verlaufskurven miteinander in Verbindung gebracht werden. 

In [ ]:
dt = datetime.today()
dt = datetime(dt.year, 1, 1)

times = pd.date_range(datetime(dt.year, 1, 1), datetime(dt.year+1, 1, 1), inclusive='left', freq='H', tz=tz)
solpos = pd.DataFrame(suncalc.get_position(times, long, lat))
solpos = pd.DataFrame(times).join(solpos, how='inner').loc[solpos['altitude'] > 0, :]
solpos = solpos.set_index(0)
solpos['altitude'] = np.vectorize(to_degree)(solpos['altitude'])
solpos['azimuth'] = np.vectorize(lambda v: math.degrees(v))(solpos['azimuth'])

fig, ax = plt.subplots(figsize=plt.figaspect(1/3))
points = ax.scatter(solpos['azimuth'], solpos['altitude'], s=2, c=solpos.index.dayofyear, label=None)
fig.colorbar(points)

for hour in np.unique(solpos.index.hour):
    subset = solpos.loc[solpos.index.hour == hour, :]
    height = subset['altitude']
    pos = solpos.loc[height.idxmax(), :]
    ax.text(pos['azimuth'], pos['altitude'], str(hour))
    
cmap = plt.colormaps.get_cmap('YlOrRd')
neutral_day = datetime(dt.year, 3, 21)
dates_of_interest = [datetime(dt.year-1, 12, 21), best_day['measured_on'][0].tz_localize(None).to_pydatetime(), neutral_day, datetime(dt.year, 6, 21)]
dates_of_interest.sort()
dates_of_interest[0] = dates_of_interest[0].replace(year=dt.year)

for index, date in enumerate(pd.to_datetime(dates_of_interest)):
    times = pd.date_range(date, date+pd.Timedelta('24h'), freq='1min', tz=tz)
    solpos = pd.DataFrame(suncalc.get_position(times, long, lat))
    solpos = solpos.loc[solpos['altitude'] > 0, :]
    solpos['altitude'] = np.vectorize(to_degree)(solpos['altitude'])
    solpos['azimuth'] = np.vectorize(lambda v: math.degrees(v))(solpos['azimuth'])
    label = date.strftime('%Y-%m-%d')
    color = 'grey' if date == neutral_day else cmap(0.4 + 0.2 * index)
    ax.plot(solpos['azimuth'], solpos['altitude'], label=label, color=color)
    
fig.legend(loc='center right')
ax.set_xlabel('Sonnenazimut in ° von Süden nach Westen')
ax.set_ylabel('Sonnenstand über dem Horizont in °')

plt.show()

In [ ]:
first_proper_readings = db.sql("SELECT value FROM domain_values WHERE name = 'FIRST_PROPER_READINGS_ON'").df()
Markdown(f"""
### Eigenverbrauchsanteil, Autarkiegrad und Amortisierung

*Achtung*: Aufgrund von Problemen mit dem Metering beginnen alle Statistiken die Verbrauch, Bezug und Einspeisung benutzen beginnen erst am **{datetime.strptime(first_proper_readings['value'][0],'%Y-%M-%d').strftime('%x')}**.
""")

#### Eigenverbrauchsanteil und Autarkiegrad

Klarsolar erklärt den Unterschied zwischen Eigenverbrauch und Autorkiegrad ganz schön [hier](https://klarsolar.de/unterschied-eigenverbrauch-autarkie/):
* Der Eigenverbrauchsanteil ist die Menge des selbst produzierten Solarstroms, die man direkt selbst verbraucht.
  <br>Er berechnet sich so: `(Erzeugung - Einspeisung) / Erzeugung * 100`
* Der Autarkiegrad beschreibt die Menge des Eigenverbrauchanteils im Verhältnis zum Gesamtstromverbrauch.
  <br>Er berechnit sich so: `(Erzeugung – Einspeisung) / Verbrauch * 100`


Unsere Quoten sehen aktuell - ohne große Optimierungen - so aus. Mein Ziel ist mittelfristig die Erhöhung des Eigenverbrauchgrades, weniger die absolute Autarkie:

In [ ]:
new_names = {'internal_consumption': 'Eigenverbrauchsanteil in %', 'autarchy': 'Autarkiegrad', 'year': 'Jahr'}
db.sql('SELECT * FROM v_yearly_internal_consumption_share').df().fillna(0).rename(columns=new_names).set_index('Jahr')

Auch hier finde ich die Darstellung pro Stunde aufschlussreich, zumindest hinsichtlich wie wir unser eigenes Verhalten optimieren können:

In [ ]:
df = db.sql('SELECT hour, autarchy, internal_consumption FROM v_average_internal_consumption_share_per_hour').df()
df = df.rename(columns=new_names)
df.plot(kind='bar', x='hour', xlabel='Stunde', ylabel='%', figsize=plt.figaspect(1/3))

plt.show()

#### Amortisierung

Bei der Volleinspeisung wird die gesamte von einer Photovoltaikanlage erzeugte elektrische Energie ins öffentliche Stromnetz eingespeist, anstatt sie (teilweise) für den Eigenverbrauch zu nutzen. Bei der Teileinspeisung hingegen wird ein Teil der von einer Photovoltaikanlage erzeugten Energie ins öffentliche Stromnetz eingespeist. Der verbleibende Teil wird für den Eigenverbrauch im Haushalt oder Gebäude verwendet. Die Vergütung für die Volleinspeisung ist nur noch marginal höher als für Teileinspeisung und lohnt sich nicht mehr für Anlagen in der Größenordnung wie hier. Unsere Bezugskosten sind mindestens um den Faktor 4 höher als die Einspeisevergütung.

Die Vergütungen für eine Anlage unserer Größe in Teileinspeisung werden unten dargestellt:

In [ ]:
db.sql("SELECT valid_from as 'Gültig von', valid_until as 'Gültig bis', value as 'ct/kWh' FROM v__selling_prices WHERE type = 'partial_sell'").df().fillna('-')

Das folgende Diagram präsentiert die laufende Summe unser initialen Investitionskosten plus Vergütung und Ersparnissen.

Die Amortisierung bei Volleinspeisung ist hypothetisch und basiert auf der reinen Vergütung.
Die Amortisierung bei Teileinspeisung basiert auf der Vergütung der eingespeisten Energie plus der Ersparniss durch den Eigenverbrauch (aka dem nicht Kaufen von Energie).

Alle Werte in der mit hellem Lila hinterlegte Fläche sind interpolierte Werte. Allerdings ist doch recht offensichtlich, dass sich eine in 2023 angeschaffte Anlege nicht mehr in derselben Dekade amortisieren wird.

In [ ]:
# Idea of the linear extrapolation is mostly from this great answer by tmthydvnprt
# https://stackoverflow.com/a/35959909/1547989

df = db.sql('SELECT month, full_export, partial_export FROM v_amortization').df()
df.set_index(['month'], inplace=True)

_, ax = plt.subplots(figsize=plt.figaspect(1/3))

legend = ['Hypothetische Volleinspeisung', 'Teileinspeisung', 'Break even']
# Memorize last entry
last_month = df.index[len(df)-1]
first_extrapolated_month = None

if len(df) < 120:    
    num_months_extrapolation = 120 - len(df)
    
    # Extend the index
    df = pd.DataFrame(data=df, index=pd.date_range(
            start=df.index[0],
            periods=len(df.index) + num_months_extrapolation,
            freq='MS'
        )
    )
    
    first_extrapolated_month = last_month + pd.DateOffset(months=1);
    # Duplicate the first extrapolated month to have enough supporting points
    df.loc[first_extrapolated_month] = df.loc[last_month]
    
    di = df.index
    df = df.reset_index(drop=True)
    
    # Function to curve fit to the data
    def func(x, a, b):
        return a * x + b
    
    # Initial parameter guess, just to kick off the optimization
    guess = (1, 1)
    
    # Create copy of data to remove NaNs for curve fitting
    fit_df = df.dropna()
    
    # Place to store function parameters for each column
    col_params = {}
    
    # Curve fit each column
    for col in fit_df.columns:
        # Get x & y
        x = fit_df.index.astype(float).values
        y = fit_df[col].values
        # Curve fit column and get curve parameters
        params = curve_fit(func, x, y, guess)
        # Store optimized parameters
        col_params[col] = params[0]
    
    # Extrapolate each column
    for col in df.columns:
        # Get the index values for NaNs in the column
        x = df[pd.isnull(df[col])].index.astype(float).values
        # Extrapolate those points with the fitted function
        df[col][x] = func(x, *col_params[col])
    
    df.index = di
    legend = legend + ['Tatsächliche Daten', 'Extrapolierte Werte']


df.plot(ax=ax)
ax.axhline(color='green')
if first_extrapolated_month != None:
    ax.axvspan(df.index[0], first_extrapolated_month, color='green', alpha=0.1)
    ax.axvspan(first_extrapolated_month, df.index[-1], color='purple', alpha=0.05)
ax.legend(legend , loc='lower right')

plt.xlabel('Zeit in Betrieb')
plt.ylabel('Aktuelle Investition bzw akkumulierter ROI in €')
plt.ylim(top=5000)

plt.show()

Photovoltaik in Deutschland mit einer Anlage dieser Größe ist kein Mittel um schnell reich zu werden. Eher, um langsamer arm zu werden wie mein Freund Oli es ausdrückte. Daher zum Abschluss wieder ein ähnliches Diagram wie in seinem Blog. Das Diagram zeigt die jährlich akkumulierten Energiekosten in unserem Haus. Die erste Kurve ohne den Nutzen und die Ersparnisse der Photovoltaikanlage zu betrachten, die zweite Kurve bilanziert die tatsächlich angefallen Verbrauchskosten mit den Ersparnissen und Vergütungen. Falls wir am Ende eines vollen Jahres die null mit dieser Kurve erreichen, bin ich zufrieden. Negative Werte bedeuten entsprechen einer realen Vergütung.

In [ ]:
df = db.sql('SELECT * FROM v_accumulated_yearly_energy_costs').df()

_, ax = plt.subplots(figsize=plt.figaspect(1/3))

df.plot(x='month', ax = ax, xlabel='Monat', ylabel='EUR')
ax.axhline(color='green', linestyle="--")
ax.legend(['Hypothetische Energiekosten ohne PV', 'Tatsächliche Stromkosten', "Hehres Ziel"])

plt.show()